In [2]:
# 라이브러리 import
import pandas as pd
import numpy as np

### Product 파일 전처리

#### 카테고리 코드 전처리

In [3]:
# product 파일 불러오기
prd = pd.read_csv('sampled_products.csv', encoding = 'utf-8')

# 카테고리 재분류 csv 파일 불러오기
category1 = pd.read_csv('category1.csv', encoding = 'utf-8')
category2 = pd.read_csv('category2.csv', encoding = 'utf-8')
category3 = pd.read_csv('category3.csv', encoding = 'utf-8')

In [4]:
# product 파일에 merge
prd_ctg = pd.merge(prd, category1, left_on = 'category1_code',  right_on = 'c1_old', how = 'left')
prd_ctg = pd.merge(prd_ctg, category2, left_on = 'category2_code', right_on = 'c2_old', how = 'left')
prd_ctg = pd.merge(prd_ctg, category3, left_on = 'category3_code', right_on = 'c3_old', how = 'left')

In [5]:
# 카테고리 대분류, 중분류, 소분류 전처리
prd_ctg.loc[prd_ctg['c1_old'].notnull(), 'category1_name'] = prd_ctg.loc[prd_ctg['c1_old'].notnull(), 'c1_name']
prd_ctg.loc[prd_ctg['c1_old'].notnull(), 'category1_code'] = prd_ctg.loc[prd_ctg['c1_old'].notnull(), 'c1_new']
prd_ctg.loc[prd_ctg['c2_old'].notnull(), 'category2_name'] = prd_ctg.loc[prd_ctg['c2_old'].notnull(), 'c2_name']
prd_ctg.loc[prd_ctg['c2_old'].notnull(), 'category2_code'] = prd_ctg.loc[prd_ctg['c2_old'].notnull(), 'c2_new']
prd_ctg.loc[prd_ctg['c3_old'].notnull(), 'category3_name'] = prd_ctg.loc[prd_ctg['c3_old'].notnull(), 'c3_name']
prd_ctg.loc[prd_ctg['c2_old'].notnull(), 'category3_code'] = prd_ctg.loc[prd_ctg['c3_old'].notnull(), 'c3_new']

In [6]:
# 컬럼 정리
prd_ctg.drop(columns = ['c1_old', 'c1_new', 'c1_name', 'c2_old', 'c2_new', 'c2_name', 'c3_old', 'c3_new', 'c3_name'], inplace = True)

#### 카테고리 간 미스 매치 코드, 이름 전처리

In [7]:
prd_ctg_2 = pd.read_csv('ctg_preprocessing.csv', encoding = 'utf-8', sep = '|')

In [8]:
prd_ctg_2 = pd.merge(prd_ctg, prd_ctg_2, left_on = 'category3_code', right_on = 'category3_code', how = 'left')

In [9]:
# 카테고리 대분류, 중분류 코드&이름 전처리
prd_ctg_2.loc[prd_ctg_2['category1_name_y'].notnull(), 'category1_code_x'] = prd_ctg_2.loc[prd_ctg_2['category1_name_y'].notnull(), 'category1_code_y']
prd_ctg_2.loc[prd_ctg_2['category1_name_y'].notnull(), 'category1_name_x'] = prd_ctg_2.loc[prd_ctg_2['category1_name_y'].notnull(), 'category1_name_y']
prd_ctg_2.loc[prd_ctg_2['category2_name_y'].notnull(), 'category2_code_x'] = prd_ctg_2.loc[prd_ctg_2['category2_name_y'].notnull(), 'category2_code_y']
prd_ctg_2.loc[prd_ctg_2['category2_name_y'].notnull(), 'category2_name_x'] = prd_ctg_2.loc[prd_ctg_2['category2_name_y'].notnull(), 'category2_name_y']

In [10]:
# 컬럼 정리
prd_ctg_2.drop(columns = ['category1_name_y', 'category1_code_y', 'category2_name_y', 'category2_code_y', 'category3_name_y'], inplace = True)
prd_ctg_2.columns = ['item_no', 'item_name', 'image_name', 'price', 'category1_code',
       'category1_name', 'category2_code', 'category2_name',
       'category3_code', 'category3_name', 'brand_no', 'brand_name']

#### 브랜드 이름 전처리

In [11]:
prd_ctg_2.loc[prd_ctg_2['brand_no'] == 2551, 'brand_name'] = '눌'

In [12]:
prd_ctg_2.loc[prd_ctg_2['item_name'] == '그린티 모이스트 파하 거즈 필링(30매)', 'brand_name'] = '네오젠'
prd_ctg_2.loc[prd_ctg_2['item_name'] == 'EMF 전자파차단 침대 거실 프리미엄 전기요', 'brand_name'] = '한스갤러리'
prd_ctg_2.loc[prd_ctg_2['item_name'] == '[프리미엄침구] 시원한 시어서커/인견 여름이불 모음', 'brand_name'] = '오르다'
prd_ctg_2.loc[prd_ctg_2['brand_name'] == '네오젠', 'brand_no'] = 7980
prd_ctg_2.loc[prd_ctg_2['brand_name'] == '오르다', 'brand_no'] = 12647
prd_ctg_2.loc[prd_ctg_2['brand_name'] == '한스갤러리', 'brand_no'] = 11456

In [13]:
del prd_ctg
del prd

In [14]:
prd_ctg_2.to_csv('product_processed.csv', encoding = 'utf-8', index = False)

### Event 파일 전처리

#### 이벤트 로그 중 'session_id', 'event_timestamp', 'event_name', 'user_no', 'item_no' 중복 행 제거

In [15]:
# event 파일 불러오기
evt = pd.read_csv('sampled_events.csv', encoding = 'utf-8')

In [16]:
# 'session_id', 'event_timestamp', 'event_name', 'user_no', 'item_no' 중복 행 제거
evt.drop_duplicates(subset = ['session_id', 'event_timestamp', 'event_name', 'user_no', 'item_no'], inplace = True)

In [85]:
evt.to_csv('event_processed.csv', encoding = 'utf-8', index = False)

### User 파일 전처리

In [87]:
# user 파일 불러오기
user = pd.read_csv('sampled_users.csv', encoding = 'utf-8')

In [99]:
# Null 값 제거
user.dropna(inplace = True)

#### age 컬럼 추가

In [100]:
# age 컬럼 생성
user['age'] = 2022 - user.birth_date.str[:4].astype('int32') + 1
# 만 14세 미만 전처리
user.drop(user[user['age'] < 15].index, inplace = True)

#### 성별 0, 1 구분 컬럼 추가

In [110]:
# F, M 컬럼 생성
user.loc[user['gender'] == 'F', 'F'] = '1'
user.loc[user['gender'] == 'F', 'M'] = '0'
user.loc[user['gender'] == 'M', 'M'] = '1'
user.loc[user['gender'] == 'M', 'F'] = '0'

In [143]:
user.to_csv('user_processed', encoding = 'utf-8', index = False)